In [1]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from time import sleep
from datetime import datetime

In [12]:
# para probar usaremos un archivo menos actualizado
df = pd.read_excel('prueba(2).xlsx')

In [13]:
df

,Urls,Oferta,Empresa,Funcion,Aptitudes,Requisito,Jornada,Contrato,Salario,Fecha_de_publicacion,Ubicacion,Idiomas,Texto,Teletrabajo o Presencial,Experiencia
0,https://www.tecnoempleo.com/software-engineeri...,Software Engineering Manager,Solera,Analista Programador,Java REST API Git CI/CD Hibernate Docker Cloud...,Ingeniero Superior,Jornada completa,Indefinido,NaN,12/04/2025,Alcobendas - Madrid - España,Inglés (Alto),Descripción de la oferta de empleo 7+ years of...,NaN,Más de 5 años
1,https://www.tecnoempleo.com/front-developer-se...,Front Developer Senior,Mytic Innova,Analista Programador,Java Angular SQL SCRUM Git Html CSS Bootstrap ...,Ingeniero Superior,Jornada completa,Indefinido,27000 33000,11/04/2025,Hospitalet de Llobregat - Barcelona,Inglés (Alto),Descripción de la oferta de empleo Mytic Innov...,Híbrido,2 años
2,https://www.tecnoempleo.com/programador-fullst...,Programador FullStack,Dev&del,Analista Programador,java javascript spring angular react html css ...,FP2/Grado Superior,Jornada completa,Indefinido,NaN,11/04/2025,Madrid,Inglés (Alto),Descripción de la oferta de empleo ¿Eres una p...,Híbrido,3 años
3,https://www.tecnoempleo.com/analista-programad...,Analista Programador en PHP,Codicert,Programador,php7 phpunit ajax composer poo psr4 svn doctrine,FP2/Grado Superior,Jornada completa,Temporal,30000 30000,11/04/2025,Gzira - Malta,Inglés (Medio),Descripción de la oferta de empleo ¡Únete a nu...,Presencial,2 años
4,https://www.tecnoempleo.com/docente-online-int...,Docente online Inteligencia Artificial,EIP Escuela Internacional Posgrados,Formador,Azure Zapier Chat GPT Make Monica Gemini,FP2/Grado Superior,Media jornada,Temporal,24 24,11/04/2025,Teletrabajo,NaN,Descripción de la oferta de empleo ¡Únete a nu...,100% remoto,1 año
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3990,https://www.tecnoempleo.com/administrador-sist...,Administrador Sistemas Linux,Stratio,Administrador,linux docker ansible python,FP2/Grado Superior,Intensiva Mañana,Indefinido,33000 42000,16/05/2018,Pozuelo De Alarcón - Madrid España,NaN,Descripción de la oferta de empleo En Stratio ...,NaN,3 años
3991,https://www.tecnoempleo.com/desarrolladores-ja...,Desarrolladores Java,Stratio,Analista Programador,java spring boot Spring Cloud,Ingeniero Técnico,Jornada completa,Indefinido,NaN,08/11/2017,Madrid - España,NaN,Descripción de la oferta de empleo Desde Strat...,NaN,3 años
3992,https://www.tecnoempleo.com/arquitecto-fullsta...,Arquitecto Fullstack,Stratio,Arquitecto TIC,jee java oracle weblogic eclipse,Ingeniero Superior,Jornada completa,Indefinido,30000 36000,25/10/2017,Madrid - España,NaN,Descripción de la oferta de empleo Experiencia...,NaN,Más de 5 años
3993,https://www.tecnoempleo.com/spark-engineer-mad...,Spark Engineer,Stratio,Arquitecto TIC,Spark Hadoop Cassandra Mongo DB Hbase,Ingeniero Técnico,Jornada completa,Indefinido,NaN,03/10/2017,Pozuelo - Madrid - España,Español,Descripción de la oferta de empleo Más de 3 añ...,NaN,3 años


In [20]:
#FUNCION PARA GENERAR LOS URLS NUEVOS
#ESTA FUNCION HA DE RECIBIR LA ENTRADA DE UN DF DE URLS
def nuevas_ofertas (df_urls):
    
    reps = int(BeautifulSoup(requests.get('https://www.tecnoempleo.com/ofertas-trabajo/?pagina=1').text, "html.parser").find("h1").text.split()[0].replace(".",""))
    links = [f"https://www.tecnoempleo.com/ofertas-trabajo/?pagina={i+1}" for i in range(reps//30+1)]
    
    nuevo = []
    urls = []
    oferta = []
    empresa = []
    
    for i in links:
        response = requests.get(i)
        soup = BeautifulSoup(response.text, "html.parser")
        for j in soup.find_all("div", class_ = "row fs--15"):
            url = j.find("a")["href"]
            if url not in df_urls.values:
                nuevo.append(url)
                try:
                    oferta.append(j.find("h3").find("a")["title"])
                except:
                    oferta.append(np.nan)
                try:
                    empresa.append(j.find("a", class_ = "text-primary link-muted").text)
                except:
                    empresa.append(np.nan)
                
            
    return pd.DataFrame({"Urls": nuevo, "Oferta": oferta, "Empresa": empresa})
    
df_nuevo = nuevas_ofertas(df["Urls"])

In [23]:
df_nuevo

,Urls,Oferta,Empresa
0,https://www.tecnoempleo.com/senior-backend-dev...,Senior Backend Developer (.NET/ Go/ PHP),WebBeds
1,https://www.tecnoempleo.com/ops-tools-senior-d...,Ops Tools Senior DevOps Engineer,Axiom Software Solutions
2,https://www.tecnoempleo.com/especialista-en-pr...,Especialista en Pruebas de Rendimiento,Grupo NS
3,https://www.tecnoempleo.com/software-quality-e...,Software Quality Engineer / Test Analyst,Almato Iberia
4,https://www.tecnoempleo.com/senior-java-develo...,Senior Java Developer,Almato Iberia
...,...,...,...
176,https://www.tecnoempleo.com/java-senior-softwa...,Java Senior Software Engineer,Page Personnel
177,https://www.tecnoempleo.com/tecnico-service-de...,Técnico/a Service Desk Logístico (B2),Serem
178,https://www.tecnoempleo.com/analista-programad...,Analista programador/a (IA Python Node JS),Krell Consulting & Training
179,https://www.tecnoempleo.com/ingeniero-simulaci...,Ingeniero/a de simulación C/C++,GMV


In [26]:
def extracción_nurls (df_urls):
    
    fecha_de_publicacion = []
    fecha_extraccion = []
    aptitudes = []
    texto = []
    funcion = []
    requisito = []
    salario = []
    contrato = []
    jornada = []
    idiomas = []
    ubicacion = []
    experiencia = []
    
    for i in df_urls:
        
        response = requests.get(i)
        soup = BeautifulSoup(response.text, "html.parser")
        numeros = []
        formacion = False
        contra = False
        jorn = False
        idiom = False
        puesto = False
        ubi = False
        text = False
        exp = False
        # EXTRACCION DE DATOS
        # Fecha de oferta
        try:
            fecha_de_publicacion.append(soup.find("span", class_ = "ml-4").text.strip()[:10])
        except:
            fecha_de_publicacion.append(np.nan)

        # Momento de extracción
        fecha_extraccion.append(datetime.now())
    
        # Aptitudes
        try:
            aptitudes.append(soup.find("div", class_= "pl--12 pr--12").text.split())
        except:
            aptitudes.append(np.nan)
            
        # Descripción de la oferta
        for k in soup.find_all("div", class_ = "mt-4"):
            if k.find("p", class_ = "fs--16 text-gray-800"):
                texto.append(" ".join(k.text.split()).strip())
                text = True
                break
        if not text:
            texto.append(np.nan)
    
        # Funcion o puesto de trabajo
        for k in soup.find_all("li", class_ = "list-item clearfix border-bottom py-2"):
            if k.find("i", class_ = "fi fi-users"):
                funcion.append(" ".join([" ".join(i.replace("Funciones", "").split()) for i in (k.text.split())]).strip())
                puesto = True
                break
        if not puesto:
            funcion.append(np.nan)
            
        # Requisitos min
        for k in soup.find_all("div", class_ = "d-flex py-2"):
            if k.find("i", class_ = "fi fi-graduate"):
                requisito.append((k.text.split(":")[1]).strip())
                formacion = True
                break
        if not formacion:
            requisito.append(np.nan)
    
        # Salario min max
        for l in soup.find_all("li", class_ = "list-item clearfix border-bottom py-2"):
            if l.find("i", class_="fi fi-plus"):
                for k in l.text.strip().replace(".","")[:-1].split():
                    if k.isdigit():
                        numeros.append(k)
        salario.append(" ".join(numeros) if numeros else np.nan)
    
        # Tipo de contrato
        for l in soup.find_all("li", class_ = "list-item clearfix border-bottom py-2"):
            if l.find("i", class_ = "fi fi-task-list"):
                contrato.append(" ".join([" ".join(i.replace("Tipo", "").replace("contrato", "").split()) for i in (l.text.split())]).strip())
                contra = True
                break
        if not contra:
            contrato.append(np.nan)
    
        # Jornada (completa, media,etc)
        for l in soup.find_all("li", class_ = "list-item clearfix border-bottom py-2"):
            if l.find("i", class_ = "fi fi-calendar"):
                jornada.append(" ".join(l.text.split()[:2]).strip())
                jorn = True
                break
        if not jorn:
            jornada.append(np.nan)
            
        # Idiomas requeridos
        for l in soup.find_all("div", class_ = "d-flex py-2"):
            if l.find("i", class_ = "fi fi-globe"):
                idiomas.append(" ".join((l.text.replace("|", "").split(":")[1]).split()))
                idiom = True
                break
        if not idiom:
            idiomas.append(np.nan)
    
        # Ubi job
        for k in soup.find_all("li", class_ = "list-item clearfix border-bottom py-2"):
            if k.find("i", class_ = "fi fi-pin"):
                ubicacion.append(" ".join(k.text.split()).strip())
                ubi = True
                break
        if not ubi:
            ubicacion.append(np.nan)
            
        # Experiencia (años)
        for k in soup.find_all("li", class_ = "list-item clearfix border-bottom py-2"):
            if k.find("i", class_ = "fi fi-shield-ok"):
                experiencia.append(" ".join(k.text.split()).replace("Experiencia", "").strip())
                exp = True
                break
        if not exp:
            experiencia.append(np.nan)
        
    sleep(1)

    return pd.DataFrame({"Funcion": funcion,"Aptitudes": aptitudes,
                         "Requisito": requisito,"Jornada": jornada,
                         "Contrato": contrato,"Salario": salario,
                         "Fecha_de_publicacion": fecha_de_publicacion,"Ubicacion": ubicacion,
                         "Idiomas": idiomas,"Texto": texto,
                         "Experiencia": experiencia,"Fecha_extraccion": fecha_extraccion})

df_nuevo1 = extracción_nurls (df_nuevo["Urls"])
df_nuevo1

,Funcion,Aptitudes,Requisito,Jornada,Contrato,Salario,Fecha_de_publicacion,Ubicacion,Idiomas,Texto,Experiencia,Fecha_extraccion
0,Programador,"[C#, Golang, PHP, git, kubernetes, api, data, ...",Sin estudios,Jornada completa,Indefinido,NaN,17/04/2025,Palma - Baleares (Híbrido),Inglés (Medio),Descripción de la oferta de empleo How will yo...,3 años,2025-04-17 15:44:07.892294
1,DevOps,"[AWS, SAP, Ansible, Chef, Terraform, ArgoCD]",NaN,Jornada completa,Indefinido,NaN,17/04/2025,100% En remoto,NaN,Descripción de la oferta de empleo JD: Contrac...,Más de 5 años,2025-04-17 15:44:08.055256
2,Tester,"[java, servicenow, ALM, loadrunner, dynatrace,...",FP2/Grado Superior,Jornada completa,Indefinido,24000 30000,17/04/2025,100% En remoto,NaN,Descripción de la oferta de empleo En Grupo NS...,3 años,2025-04-17 15:44:08.220538
3,Analista,"[GO, ISTQB]",Ingeniero Superior,Media jornada,Indefinido,30000 45000,17/04/2025,100% En remoto,Alemán (Excelente) Inglés (Alto),Descripción de la oferta de empleo En Almato I...,Más de 5 años,2025-04-17 15:44:08.409329
4,Analista Programador,"[java, sql, sap, angular, spring]",FP2/Grado Superior,Jornada completa,Indefinido,33000 36000,17/04/2025,100% En remoto,Alemán (Alto) Inglés (Alto),Descripción de la oferta de empleo En nuestra ...,Más de 5 años,2025-04-17 15:44:08.573328
...,...,...,...,...,...,...,...,...,...,...,...,...
176,Programador,"[java, API, Springboot]",NaN,Jornada completa,Indefinido,54000 72000,14/04/2025,Barcelona (Híbrido),NaN,Descripción de la oferta de empleo Perfil busc...,Más de 5 años,2025-04-17 15:44:38.581235
177,Helpdesk,[SQL],Grado Medio,Jornada completa,Indefinido,27000 30000,14/04/2025,Madrid (Híbrido),Inglés (Alto),Descripción de la oferta de empleo En Serem es...,3 años,2025-04-17 15:44:38.752576
178,Analista Programador,"[Node, js, TypeScript, JavaScript, Python]",NaN,Jornada completa,Indefinido,NaN,28/03/2025,100% En remoto,NaN,Descripción de la oferta de empleo ### **Descr...,2 años,2025-04-17 15:44:38.921933
179,Programador,"[C, C++, Linux, OS, Git, Eclipse, Visual, Studio]",NaN,Jornada completa,Indefinido,NaN,21/03/2025,Madrid - España,NaN,Descripción de la oferta de empleo JOB.TITLE S...,1 año,2025-04-17 15:44:39.082613


In [28]:
df_final = pd.concat([df_nuevo, df_nuevo1], axis = 1)
df_final

,Urls,Oferta,Empresa,Funcion,Aptitudes,Requisito,Jornada,Contrato,Salario,Fecha_de_publicacion,Ubicacion,Idiomas,Texto,Experiencia,Fecha_extraccion
0,https://www.tecnoempleo.com/senior-backend-dev...,Senior Backend Developer (.NET/ Go/ PHP),WebBeds,Programador,"[C#, Golang, PHP, git, kubernetes, api, data, ...",Sin estudios,Jornada completa,Indefinido,NaN,17/04/2025,Palma - Baleares (Híbrido),Inglés (Medio),Descripción de la oferta de empleo How will yo...,3 años,2025-04-17 15:44:07.892294
1,https://www.tecnoempleo.com/ops-tools-senior-d...,Ops Tools Senior DevOps Engineer,Axiom Software Solutions,DevOps,"[AWS, SAP, Ansible, Chef, Terraform, ArgoCD]",NaN,Jornada completa,Indefinido,NaN,17/04/2025,100% En remoto,NaN,Descripción de la oferta de empleo JD: Contrac...,Más de 5 años,2025-04-17 15:44:08.055256
2,https://www.tecnoempleo.com/especialista-en-pr...,Especialista en Pruebas de Rendimiento,Grupo NS,Tester,"[java, servicenow, ALM, loadrunner, dynatrace,...",FP2/Grado Superior,Jornada completa,Indefinido,24000 30000,17/04/2025,100% En remoto,NaN,Descripción de la oferta de empleo En Grupo NS...,3 años,2025-04-17 15:44:08.220538
3,https://www.tecnoempleo.com/software-quality-e...,Software Quality Engineer / Test Analyst,Almato Iberia,Analista,"[GO, ISTQB]",Ingeniero Superior,Media jornada,Indefinido,30000 45000,17/04/2025,100% En remoto,Alemán (Excelente) Inglés (Alto),Descripción de la oferta de empleo En Almato I...,Más de 5 años,2025-04-17 15:44:08.409329
4,https://www.tecnoempleo.com/senior-java-develo...,Senior Java Developer,Almato Iberia,Analista Programador,"[java, sql, sap, angular, spring]",FP2/Grado Superior,Jornada completa,Indefinido,33000 36000,17/04/2025,100% En remoto,Alemán (Alto) Inglés (Alto),Descripción de la oferta de empleo En nuestra ...,Más de 5 años,2025-04-17 15:44:08.573328
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176,https://www.tecnoempleo.com/java-senior-softwa...,Java Senior Software Engineer,Page Personnel,Programador,"[java, API, Springboot]",NaN,Jornada completa,Indefinido,54000 72000,14/04/2025,Barcelona (Híbrido),NaN,Descripción de la oferta de empleo Perfil busc...,Más de 5 años,2025-04-17 15:44:38.581235
177,https://www.tecnoempleo.com/tecnico-service-de...,Técnico/a Service Desk Logístico (B2),Serem,Helpdesk,[SQL],Grado Medio,Jornada completa,Indefinido,27000 30000,14/04/2025,Madrid (Híbrido),Inglés (Alto),Descripción de la oferta de empleo En Serem es...,3 años,2025-04-17 15:44:38.752576
178,https://www.tecnoempleo.com/analista-programad...,Analista programador/a (IA Python Node JS),Krell Consulting & Training,Analista Programador,"[Node, js, TypeScript, JavaScript, Python]",NaN,Jornada completa,Indefinido,NaN,28/03/2025,100% En remoto,NaN,Descripción de la oferta de empleo ### **Descr...,2 años,2025-04-17 15:44:38.921933
179,https://www.tecnoempleo.com/ingeniero-simulaci...,Ingeniero/a de simulación C/C++,GMV,Programador,"[C, C++, Linux, OS, Git, Eclipse, Visual, Studio]",NaN,Jornada completa,Indefinido,NaN,21/03/2025,Madrid - España,NaN,Descripción de la oferta de empleo JOB.TITLE S...,1 año,2025-04-17 15:44:39.082613


In [30]:
## De esta manera crearemos la columna de Teletrabajo o Presencial con la de Ubicacion.
def ext_teletrabajo(df_ubicacion):
    teletrabajo = []
    for i in df_ubicacion.values:
        if not pd.isna(i):
            if "Presencial" in i:
                teletrabajo.append("Presencial")
                
            elif "remoto" in i:
                teletrabajo.append("100% remoto")
                
            elif "(Híbrido)" in i:
                teletrabajo.append("Híbrido")
                
            else:
                teletrabajo.append(np.nan)
        else:
            teletrabajo.append(i)
        
    return teletrabajo

df_final["Teletrabajo o Presencial"] = ext_teletrabajo(df_final["Ubicacion"])
df_final

,Urls,Oferta,Empresa,Funcion,Aptitudes,Requisito,Jornada,Contrato,Salario,Fecha_de_publicacion,Ubicacion,Idiomas,Texto,Experiencia,Fecha_extraccion,Teletrabajo o Presencial
0,https://www.tecnoempleo.com/senior-backend-dev...,Senior Backend Developer (.NET/ Go/ PHP),WebBeds,Programador,"[C#, Golang, PHP, git, kubernetes, api, data, ...",Sin estudios,Jornada completa,Indefinido,NaN,17/04/2025,Palma - Baleares (Híbrido),Inglés (Medio),Descripción de la oferta de empleo How will yo...,3 años,2025-04-17 15:44:07.892294,Híbrido
1,https://www.tecnoempleo.com/ops-tools-senior-d...,Ops Tools Senior DevOps Engineer,Axiom Software Solutions,DevOps,"[AWS, SAP, Ansible, Chef, Terraform, ArgoCD]",NaN,Jornada completa,Indefinido,NaN,17/04/2025,100% En remoto,NaN,Descripción de la oferta de empleo JD: Contrac...,Más de 5 años,2025-04-17 15:44:08.055256,100% remoto
2,https://www.tecnoempleo.com/especialista-en-pr...,Especialista en Pruebas de Rendimiento,Grupo NS,Tester,"[java, servicenow, ALM, loadrunner, dynatrace,...",FP2/Grado Superior,Jornada completa,Indefinido,24000 30000,17/04/2025,100% En remoto,NaN,Descripción de la oferta de empleo En Grupo NS...,3 años,2025-04-17 15:44:08.220538,100% remoto
3,https://www.tecnoempleo.com/software-quality-e...,Software Quality Engineer / Test Analyst,Almato Iberia,Analista,"[GO, ISTQB]",Ingeniero Superior,Media jornada,Indefinido,30000 45000,17/04/2025,100% En remoto,Alemán (Excelente) Inglés (Alto),Descripción de la oferta de empleo En Almato I...,Más de 5 años,2025-04-17 15:44:08.409329,100% remoto
4,https://www.tecnoempleo.com/senior-java-develo...,Senior Java Developer,Almato Iberia,Analista Programador,"[java, sql, sap, angular, spring]",FP2/Grado Superior,Jornada completa,Indefinido,33000 36000,17/04/2025,100% En remoto,Alemán (Alto) Inglés (Alto),Descripción de la oferta de empleo En nuestra ...,Más de 5 años,2025-04-17 15:44:08.573328,100% remoto
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176,https://www.tecnoempleo.com/java-senior-softwa...,Java Senior Software Engineer,Page Personnel,Programador,"[java, API, Springboot]",NaN,Jornada completa,Indefinido,54000 72000,14/04/2025,Barcelona (Híbrido),NaN,Descripción de la oferta de empleo Perfil busc...,Más de 5 años,2025-04-17 15:44:38.581235,Híbrido
177,https://www.tecnoempleo.com/tecnico-service-de...,Técnico/a Service Desk Logístico (B2),Serem,Helpdesk,[SQL],Grado Medio,Jornada completa,Indefinido,27000 30000,14/04/2025,Madrid (Híbrido),Inglés (Alto),Descripción de la oferta de empleo En Serem es...,3 años,2025-04-17 15:44:38.752576,Híbrido
178,https://www.tecnoempleo.com/analista-programad...,Analista programador/a (IA Python Node JS),Krell Consulting & Training,Analista Programador,"[Node, js, TypeScript, JavaScript, Python]",NaN,Jornada completa,Indefinido,NaN,28/03/2025,100% En remoto,NaN,Descripción de la oferta de empleo ### **Descr...,2 años,2025-04-17 15:44:38.921933,100% remoto
179,https://www.tecnoempleo.com/ingeniero-simulaci...,Ingeniero/a de simulación C/C++,GMV,Programador,"[C, C++, Linux, OS, Git, Eclipse, Visual, Studio]",NaN,Jornada completa,Indefinido,NaN,21/03/2025,Madrid - España,NaN,Descripción de la oferta de empleo JOB.TITLE S...,1 año,2025-04-17 15:44:39.082613,NaN


In [31]:
df_final = df_final[df.columns]
df_final

,Urls,Oferta,Empresa,Funcion,Aptitudes,Requisito,Jornada,Contrato,Salario,Fecha_de_publicacion,Ubicacion,Idiomas,Texto,Teletrabajo o Presencial,Experiencia
0,https://www.tecnoempleo.com/senior-backend-dev...,Senior Backend Developer (.NET/ Go/ PHP),WebBeds,Programador,"[C#, Golang, PHP, git, kubernetes, api, data, ...",Sin estudios,Jornada completa,Indefinido,NaN,17/04/2025,Palma - Baleares (Híbrido),Inglés (Medio),Descripción de la oferta de empleo How will yo...,Híbrido,3 años
1,https://www.tecnoempleo.com/ops-tools-senior-d...,Ops Tools Senior DevOps Engineer,Axiom Software Solutions,DevOps,"[AWS, SAP, Ansible, Chef, Terraform, ArgoCD]",NaN,Jornada completa,Indefinido,NaN,17/04/2025,100% En remoto,NaN,Descripción de la oferta de empleo JD: Contrac...,100% remoto,Más de 5 años
2,https://www.tecnoempleo.com/especialista-en-pr...,Especialista en Pruebas de Rendimiento,Grupo NS,Tester,"[java, servicenow, ALM, loadrunner, dynatrace,...",FP2/Grado Superior,Jornada completa,Indefinido,24000 30000,17/04/2025,100% En remoto,NaN,Descripción de la oferta de empleo En Grupo NS...,100% remoto,3 años
3,https://www.tecnoempleo.com/software-quality-e...,Software Quality Engineer / Test Analyst,Almato Iberia,Analista,"[GO, ISTQB]",Ingeniero Superior,Media jornada,Indefinido,30000 45000,17/04/2025,100% En remoto,Alemán (Excelente) Inglés (Alto),Descripción de la oferta de empleo En Almato I...,100% remoto,Más de 5 años
4,https://www.tecnoempleo.com/senior-java-develo...,Senior Java Developer,Almato Iberia,Analista Programador,"[java, sql, sap, angular, spring]",FP2/Grado Superior,Jornada completa,Indefinido,33000 36000,17/04/2025,100% En remoto,Alemán (Alto) Inglés (Alto),Descripción de la oferta de empleo En nuestra ...,100% remoto,Más de 5 años
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176,https://www.tecnoempleo.com/java-senior-softwa...,Java Senior Software Engineer,Page Personnel,Programador,"[java, API, Springboot]",NaN,Jornada completa,Indefinido,54000 72000,14/04/2025,Barcelona (Híbrido),NaN,Descripción de la oferta de empleo Perfil busc...,Híbrido,Más de 5 años
177,https://www.tecnoempleo.com/tecnico-service-de...,Técnico/a Service Desk Logístico (B2),Serem,Helpdesk,[SQL],Grado Medio,Jornada completa,Indefinido,27000 30000,14/04/2025,Madrid (Híbrido),Inglés (Alto),Descripción de la oferta de empleo En Serem es...,Híbrido,3 años
178,https://www.tecnoempleo.com/analista-programad...,Analista programador/a (IA Python Node JS),Krell Consulting & Training,Analista Programador,"[Node, js, TypeScript, JavaScript, Python]",NaN,Jornada completa,Indefinido,NaN,28/03/2025,100% En remoto,NaN,Descripción de la oferta de empleo ### **Descr...,100% remoto,2 años
179,https://www.tecnoempleo.com/ingeniero-simulaci...,Ingeniero/a de simulación C/C++,GMV,Programador,"[C, C++, Linux, OS, Git, Eclipse, Visual, Studio]",NaN,Jornada completa,Indefinido,NaN,21/03/2025,Madrid - España,NaN,Descripción de la oferta de empleo JOB.TITLE S...,NaN,1 año


In [32]:
df_f = pd.concat([df, df_final], ignore_index = True)
df_f

,Urls,Oferta,Empresa,Funcion,Aptitudes,Requisito,Jornada,Contrato,Salario,Fecha_de_publicacion,Ubicacion,Idiomas,Texto,Teletrabajo o Presencial,Experiencia
0,https://www.tecnoempleo.com/software-engineeri...,Software Engineering Manager,Solera,Analista Programador,Java REST API Git CI/CD Hibernate Docker Cloud...,Ingeniero Superior,Jornada completa,Indefinido,NaN,12/04/2025,Alcobendas - Madrid - España,Inglés (Alto),Descripción de la oferta de empleo 7+ years of...,NaN,Más de 5 años
1,https://www.tecnoempleo.com/front-developer-se...,Front Developer Senior,Mytic Innova,Analista Programador,Java Angular SQL SCRUM Git Html CSS Bootstrap ...,Ingeniero Superior,Jornada completa,Indefinido,27000 33000,11/04/2025,Hospitalet de Llobregat - Barcelona,Inglés (Alto),Descripción de la oferta de empleo Mytic Innov...,Híbrido,2 años
2,https://www.tecnoempleo.com/programador-fullst...,Programador FullStack,Dev&del,Analista Programador,java javascript spring angular react html css ...,FP2/Grado Superior,Jornada completa,Indefinido,NaN,11/04/2025,Madrid,Inglés (Alto),Descripción de la oferta de empleo ¿Eres una p...,Híbrido,3 años
3,https://www.tecnoempleo.com/analista-programad...,Analista Programador en PHP,Codicert,Programador,php7 phpunit ajax composer poo psr4 svn doctrine,FP2/Grado Superior,Jornada completa,Temporal,30000 30000,11/04/2025,Gzira - Malta,Inglés (Medio),Descripción de la oferta de empleo ¡Únete a nu...,Presencial,2 años
4,https://www.tecnoempleo.com/docente-online-int...,Docente online Inteligencia Artificial,EIP Escuela Internacional Posgrados,Formador,Azure Zapier Chat GPT Make Monica Gemini,FP2/Grado Superior,Media jornada,Temporal,24 24,11/04/2025,Teletrabajo,NaN,Descripción de la oferta de empleo ¡Únete a nu...,100% remoto,1 año
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4171,https://www.tecnoempleo.com/java-senior-softwa...,Java Senior Software Engineer,Page Personnel,Programador,"[java, API, Springboot]",NaN,Jornada completa,Indefinido,54000 72000,14/04/2025,Barcelona (Híbrido),NaN,Descripción de la oferta de empleo Perfil busc...,Híbrido,Más de 5 años
4172,https://www.tecnoempleo.com/tecnico-service-de...,Técnico/a Service Desk Logístico (B2),Serem,Helpdesk,[SQL],Grado Medio,Jornada completa,Indefinido,27000 30000,14/04/2025,Madrid (Híbrido),Inglés (Alto),Descripción de la oferta de empleo En Serem es...,Híbrido,3 años
4173,https://www.tecnoempleo.com/analista-programad...,Analista programador/a (IA Python Node JS),Krell Consulting & Training,Analista Programador,"[Node, js, TypeScript, JavaScript, Python]",NaN,Jornada completa,Indefinido,NaN,28/03/2025,100% En remoto,NaN,Descripción de la oferta de empleo ### **Descr...,100% remoto,2 años
4174,https://www.tecnoempleo.com/ingeniero-simulaci...,Ingeniero/a de simulación C/C++,GMV,Programador,"[C, C++, Linux, OS, Git, Eclipse, Visual, Studio]",NaN,Jornada completa,Indefinido,NaN,21/03/2025,Madrid - España,NaN,Descripción de la oferta de empleo JOB.TITLE S...,NaN,1 año
